In [2]:
import re
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Lambda, Input, Embedding, LSTM, SpatialDropout1D
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from string import punctuation
%matplotlib inline

Using TensorFlow backend.


In [3]:
sdb = pd.read_csv("Shakespeare-plays/Shakespeare_data.csv")

In [4]:
class_names = ['comedy','tragedy','other']

In [5]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in sdb['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [6]:
all_plays = all_the_plays()

In [7]:
def get_text(play_name):
    play = sdb[sdb['Play']==play_name]
    w = play['PlayerLine']
    text = ""
    for i in w:
        text += i
        text += " "
    inp_sequences, total_words = get_sequence_of_tokens(text)
    inp_sequences[:10]
    return text

In [66]:
comedy_training = []
tragedy_training = []
history_training = []

In [9]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    corpus = corpus[:7000]
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [10]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [9]:
def get_training_text():
    comedy_training.append(get_text("Alls well that ends well"))
    comedy_training.append(get_text("As you like it"))
    comedy_training.append(get_text("Loves Labours Lost"))
    comedy_training.append(get_text("Measure for measure"))
    comedy_training.append(get_text("Merry Wives of Windsor"))
    comedy_training.append(get_text("A Midsummer nights dream"))
    comedy_training.append(get_text("Taming of the Shrew"))
    comedy_training.append(get_text("The Tempest"))
    comedy_training.append(get_text("Two Gentlemen of Verona"))
    comedy_training.append(get_text("A Winters Tale"))
    tragedy_training.append(get_text("Antony and Cleopatra"))
    tragedy_training.append(get_text("Coriolanus"))
    tragedy_training.append(get_text("Hamlet"))
    tragedy_training.append(get_text("Julius Caesar"))
    tragedy_training.append(get_text("macbeth"))
    tragedy_training.append(get_text("Othello"))
    tragedy_training.append(get_text("Timon of Athens"))
    tragedy_training.append(get_text("Titus Andronicus"))
    history_training.append(get_text("Henry IV"))
    history_training.append(get_text("Henry VI Part 1"))
    history_training.append(get_text("Henry VI Part 3"))
    history_training.append(get_text("Henry VIII"))
    history_training.append(get_text("King John"))
    history_training.append(get_text("Richard II"))

In [67]:
comedy_testing = []
tragedy_testing = []
history_testing = []

In [11]:
def get_testing_text():
    comedy_testing.append(get_text("A Comedy of Errors"))
    comedy_testing.append(get_text("Merchant of Venice"))
    comedy_testing.append(get_text("Much Ado about nothing"))
    comedy_testing.append(get_text("Twelfth Night"))
    tragedy_testing.append(get_text("Cymbeline"))
    tragedy_testing.append(get_text("King Lear"))
    tragedy_testing.append(get_text("Romeo and Juliet"))
    tragedy_testing.append(get_text("Troilus and Cressida"))
    history_testing.append(get_text("Henry V"))
    history_testing.append(get_text("Henry VI Part 3"))
    history_testing.append(get_text("Pericles"))
    history_testing.append(get_text("Richard III"))

In [68]:
get_training_text()
get_testing_text()

In [13]:
def get_shakes_text_array(text):
    w = re.split("\W*[^\'\w+\']", text)
    text_array = []
    for i in w:
        text_array.append([i])
    return text_array

In [37]:
# def create_model(max_sequence_len, total_words):
#     input_len = max_sequence_len - 1
#     model = Sequential()
    
#     # Add Input Embedding Layer
#     model.add(Embedding(total_words, 10, input_length=input_len))
    
#     # Add Hidden Layer 1 - LSTM Layer
#     model.add(LSTM(512))
#     model.add(Dropout(0.4))
    
#     # Add Output Layer
#     model.add(Dense(total_words, activation='softmax'))

#     model.compile(loss='categorical_crossentropy', optimizer='adam')
    
#     return model

# model = create_model(max_sequence_len, total_words)
# model.summary()

In [20]:
def word_counter(words):
    count_words = Counter(words)
    total_words = len(words)
    sorted_words = count_words.most_common(total_words)
    return sorted_words

In [39]:
# vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [21]:
def word_encoding(words):
    words_split = words.split('\n')
    words_int = []
    for play in words_split:
        p = [words_int[w] for w in play.split()]
        words_int.append(p)
    return words_int

In [18]:
def encoding(labels_split):
    encoded_labels = []
    for l in labels_split:
        if l == 'comedy':
            encoded_labels.append(0)
        elif l == 'tragedy':
            encoded_labels.append(1)
        else:
            encoded_labels.append(2)
    return encoded_labels

In [42]:
# np.array(encoded_labels) = encoding(labels_split)

In [73]:
cc = word_counter(comedy_training[0])

In [75]:
ccc = {w:i+1 for i, (w,c) in enumerate(cc)}

In [76]:
print(ccc)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'h': 6, 's': 7, 'n': 8, 'r': 9, 'i': 10, 'l': 11, 'd': 12, 'u': 13, 'm': 14, ',': 15, 'y': 16, 'w': 17, 'f': 18, 'c': 19, 'g': 20, 'p': 21, 'b': 22, 'v': 23, '.': 24, 'I': 25, 'k': 26, "'": 27, 'T': 28, ':': 29, 'A': 30, 'W': 31, 'E': 32, '?': 33, 'S': 34, 'H': 35, '-': 36, 'M': 37, 'F': 38, 'B': 39, 'N': 40, 'L': 41, 'O': 42, '!': 43, 'x': 44, 'R': 45, 'C': 46, 'G': 47, 'Y': 48, 'q': 49, 'P': 50, 'D': 51, 'j': 52, 'U': 53, 'K': 54, 'V': 55, 'z': 56, '[': 57, ']': 58, 'J': 59, 'Q': 60, '\t': 61}


In [1]:
def pad_features(plays_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(plays_int), seq_length), dtype = int)
    
    for i, p in enumerate(plays_int):
        play_len = len(p)
        
        if play_len <= seq_length:
            zeroes = list(np.zeros(seq_length-play_len))
            new = zeroes+p
        elif play_len > seq_length:
            new = play[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features

In [2]:
def train_test_split():
    split_frac = 0.8
    train_x = features[0:int(split_frac*len_feat)]
    train_y = encoded_labels[0:int(split_frac*len_feat)]
    remaining_x = features[int(split_frac*len_feat):]
    remaining_y = encoded_labels[int(split_frac*len_feat):]
    valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
    valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
    test_x = remaining_x[int(len(remaining_x)*0.5):]
    test_y = remaining_y[int(len(remaining_y)*0.5):]

In [2]:
def create_tensor_datasets():
    # create Tensor datasets
    train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
    test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
    # dataloaders
    batch_size = 50
    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [42]:
"""Class thanks to Samarth Agrawal"""
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [14]:
# def model_with_params():
#     # Instantiate the model w/ hyperparams
#     vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
#     output_size = 1
#     embedding_dim = 400
#     hidden_dim = 256
#     n_layers = 2
#     net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
#     print(net)
#     SentimentRNN(
#       (embedding): Embedding(74073, 400)
#       (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
#       (dropout): Dropout(p=0.3)
#       (fc): Linear(in_features=256, out_features=1, bias=True)
#       (sig): Sigmoid()
#     )

In [43]:
def training():
    # loss and optimization functions
    lr=0.001

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)


    # training params

    epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

    counter = 0
    print_every = 100
    clip=5 # gradient clipping

    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    net.train()
    # train for some number of epochs
                        
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels in train_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            inputs = inputs.type(torch.LongTensor)
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for inputs, labels in valid_loader:

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    inputs = inputs.type(torch.LongTensor)
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())
                    
                    val_h = tuple([each.data for each in val_h])

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [17]:
def testing():
    # Get test data loss and accuracy

    test_losses = [] # track loss
    num_correct = 0

    # init hidden state
    h = net.init_hidden(batch_size)

    net.eval()
    # iterate over test data
    for inputs, labels in test_loader:

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # get predicted outputs
        inputs = inputs.type(torch.LongTensor)
        output, h = net(inputs, h)

        # calculate loss
        test_loss = criterion(output.squeeze(), labels.float())
        test_losses.append(test_loss.item())

        # convert output probabilities to predicted class (0 or 1)
        pred = torch.round(output.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        num_correct += np.sum(correct)


    # -- stats! -- ##
    # avg test loss
    print("Test loss: {:.3f}".format(np.mean(test_losses)))

    # accuracy over all test data
    test_acc = num_correct/len(test_loader.dataset)
    print("Test accuracy: {:.3f}".format(test_acc))

In [45]:
def tokenize_play(test_play):
    test_play = test_play.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_play if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints


In [46]:
def predict(net, test_play, sequence_length=200):
    
    net.eval()
    
    # tokenize play
    test_ints = tokenize_play(test_play)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==0):
        print("Tiss a happy play!")
    elif(pred.item()==1):
        print("Tiss a sad play!")
    else:
        print("Tiss a play recounting history!")

In [22]:
"""Once more, with feeling"""

'Once more, with feeling'

In [23]:
def tokenizer():
    max_fatures = 2000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(data['text'].values)
    X = tokenizer.texts_to_sequences(data['text'].values)
    X = pad_sequences(X)

In [24]:
# Soon...
# embed_dim = 128
# lstm_out = 196

# model = Sequential()
# model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
# model.add(SpatialDropout1D(0.4))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

In [25]:
# Y = pd.get_dummies(data['sentiment']).values
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)

In [26]:
# batch_size = 32
# model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

In [27]:
# validation_size = 1500

# X_validate = X_test[-validation_size:]
# Y_validate = Y_test[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]
# score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
# print("score: %.2f" % (score))
# print("acc: %.2f" % (acc))

In [28]:
def letsee():
    pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
    for x in range(len(X_validate)):

        result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]

        if np.argmax(result) == np.argmax(Y_validate[x]):
            if np.argmax(Y_validate[x]) == 0:
                neg_correct += 1
            else:
                pos_correct += 1

        if np.argmax(Y_validate[x]) == 0:
            neg_cnt += 1
        else:
            pos_cnt += 1



    print("pos_acc", pos_correct/pos_cnt*100, "%")
    print("neg_acc", neg_correct/neg_cnt*100, "%")

In [29]:
# twt = ['Meetings: Because none of us is as dumb as all of us.']
# #vectorizing the tweet by the pre-fitted tokenizer instance
# twt = tokenizer.texts_to_sequences(twt)
# #padding the tweet to have exactly the same shape as `embedding_2` input
# twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
# print(twt)
# sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
# if(np.argmax(sentiment) == 0):
#     print("negative")
# elif (np.argmax(sentiment) == 1):
#     print("positive")

In [14]:
"""Third time, third time!!"""

'Third time, third time!!'

In [87]:
tk = Tokenizer()

In [33]:
# vocabulary_size = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
# print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

In [ ]:
train_array = []
for i in comedy_training:
    tk.fit_on_texts(i)
    index_list = tk.texts_to_sequences(i)
    train_array.append(pad_sequences(index_list, maxlen=50000))
for x in tragedy_training:
    train_array.append(x)
for z in history_training:
    train_array.append(z)

In [35]:
test_array = []
for i in comedy_testing:
    train_array.append(i)
for x in tragedy_testing:
    train_array.append(x)
for z in history_testing:
    train_array.append(z)

In [36]:
train_labels = []
for i in comedy_training:
    train_labels.append(0)
for x in tragedy_training:
    train_labels.append(1)
for z in history_training:
    train_labels.append(2)

In [37]:
test_labels = []
for i in comedy_testing:
    train_labels.append(0)
for x in tragedy_testing:
    train_labels.append(1)
for z in history_testing:
    train_labels.append(2)

In [38]:
X_train = train_array
X_test = test_array
y_train = train_labels
y_test = test_labels

In [40]:
max_words = 5000000
vocabulary_size = 5000
# X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [41]:
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000000, 32)       160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [85]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:], y_train[:]
X_train2, y_train2 = X_train[:], y_train[:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

AttributeError: 'str' object has no attribute 'ndim'

In [41]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])